In [4]:
from typing import List
import itertools
from copy import copy
from tqdm import tqdm
import sys

In [5]:
class SpringRow:
    def __init__(self, row: str) -> None:
        self.springs = row.split()[0]
        self.counts = [int(count_entry) for count_entry in row.split()[1].split(',')]

    def repair_springs(self) -> int:
        count_solutions = 0

        count_unknowns = self.springs.count("?")
        combinations = itertools.product(['.', '#'], repeat=count_unknowns)
        for combination in combinations:
            replacements = list(combination)
            replacement_indices = [i for i, c in enumerate(self.springs) if c == "?"]
            new_springs = list(copy(self.springs))
            for loc, replacement in zip(replacement_indices, replacements):
                new_springs[loc] = replacement
            new_springs = "".join(new_springs)
            #print("new springs", new_springs)
            if self.compare_to_counts(new_springs):
                count_solutions += 1 

        return count_solutions

    def compare_to_counts(self, adjusted_springs: str) -> bool:
        adjusted_springs = adjusted_springs.replace('.', ' ')
        damaged_parts = adjusted_springs.split()
        damaged_lengths = [len(part) for part in damaged_parts]
        #print("damaged parts", damaged_parts, "damaged_lengths", damaged_lengths)
        return damaged_lengths == self.counts


    def __repr__(self) -> str:
        return f"Springs: {self.springs} | Counts: {self.counts}"

In [6]:
def solve_part1(input_file: str) -> int:
    with open(input_file) as f:
        spring_rows = [SpringRow(line) for line in f.readlines()]
    
    sum_of_options = 0

    for spring in spring_rows:
        #print(spring)
        options = spring.repair_springs()
        #print(options)
        sum_of_options += options
    return sum_of_options

In [7]:
solve_part1('example.txt')

21

## Part 2

In [16]:
class SpringRow2:
    def __init__(self, row: str) -> None:
        self.springs = row.split()[0]
        self.counts = [int(count_entry) for count_entry in row.split()[1].split(',')]

    def fill_springs_manually(self) -> None:
        max_defective_springs = max(self.counts)
        cur_streak = 0
        # fill with '.' for every '?' over the max possible length
        for index, entry in enumerate(self.springs):
            if entry == '#':
                cur_streak += 1
            elif entry == '.':
                cur_streak = 0
            elif entry == '?' and cur_streak == max_defective_springs:
                self.springs = self.springs[:index] + '.' + self.springs[index+1:]
                cur_streak = 0
        # fill with '#' for long start and end sequences
        start_index = self.counts[0]
        start_filling = False
        for index, entry in enumerate(self.springs[:start_index]):
            if entry == '#':
                start_filling = True
            if start_filling and entry == '?':
                self.springs = self.springs[:index] + '#' + self.springs[index+1:]
        end_index = self.counts[-1]
        start_filling = False
        for index in range(len(self.springs)-1, len(self.springs)-end_index-1, -1):
            entry = self.springs[index]
            if entry == '#':
                start_filling = True
            if start_filling and entry == '?':
                self.springs = self.springs[:index] + '#' + self.springs[index+1:]


    def repair_springs(self) -> int:
        count_solutions = 0
        print("Original spring", self.springs)

        count_unknowns = self.springs.count("?")
        print("count unknowns", count_unknowns)

        print("Manual filling")
        self.fill_springs_manually()
        print(self)

        count_unknowns = self.springs.count("?")
        print("after filling count unknowns", count_unknowns)

        combinations = itertools.product(['.', '#'], repeat=count_unknowns)
        nr_existing_damages = self.springs.count('#')
        #print("all possible combinations", len(list(combinations)))
        filtered_combinations = [combination for combination in combinations if combination.count('#') == (sum(self.counts)-nr_existing_damages)]
        print("Found combinations:", len(filtered_combinations))
        # for combination in tqdm(filtered_combinations, desc="combinations", total=len(filtered_combinations), file=sys.stdout):
        #     replacements = list(combination)
        #     #print("replacements", replacements)
        #     replacement_indices = [i for i, c in enumerate(self.springs) if c == "?"]
        #     new_springs = list(copy(self.springs))
        #     for loc, replacement in zip(replacement_indices, replacements):
        #         new_springs[loc] = replacement
        #     new_springs = "".join(new_springs)
        #     #print("new springs", new_springs)
        #     if self.compare_to_counts(new_springs):
        #         count_solutions += 1 

        return count_solutions
    
    def expand_springs(self) -> None:
        new_springs = "?".join([self.springs] * 5)
        self.springs = new_springs
        new_counts = self.counts * 5
        self.counts = new_counts

    def compare_to_counts(self, adjusted_springs: str) -> bool:
        adjusted_springs = adjusted_springs.replace('.', ' ')
        damaged_parts = adjusted_springs.split()
        damaged_lengths = [len(part) for part in damaged_parts]
        #print("damaged parts", damaged_parts, "damaged_lengths", damaged_lengths)
        return damaged_lengths == self.counts


    def __repr__(self) -> str:
        return f"Springs: {self.springs} | Counts: {self.counts}"

In [17]:
def solve_part2(input_file: str) -> int:
    with open(input_file) as f:
        spring_rows = [SpringRow2(line) for line in f.readlines()]
    
    sum_of_options = 0

    for spring in tqdm(spring_rows):
        spring.expand_springs()
        #print(spring)
        options = spring.repair_springs()
        #print(options)
        sum_of_options += options
    return sum_of_options

In [18]:
solve_part2('example.txt')

  0%|          | 0/6 [00:00<?, ?it/s]

Original spring ???.###????.###????.###????.###????.###
count unknowns 19
Manual filling
Springs: ???.###.???.###.???.###.???.###.???.### | Counts: [1, 1, 3, 1, 1, 3, 1, 1, 3, 1, 1, 3, 1, 1, 3]
after filling count unknowns 15
Found combinations: 3003
Original spring .??..??...?##.?.??..??...?##.?.??..??...?##.?.??..??...?##.?.??..??...?##.
count unknowns 29
Manual filling
Springs: .??..??...?##.?.??..??...?##.?.??..??...?##.?.??..??...?##.?.??..??...?##. | Counts: [1, 1, 3, 1, 1, 3, 1, 1, 3, 1, 1, 3, 1, 1, 3]
after filling count unknowns 29


 17%|█▋        | 1/6 [00:19<01:39, 19.91s/it]


KeyboardInterrupt: 